# Model Compare

Comparing the results of the models

In [ ]:
from pathlib import Path
import json

In [ ]:
MODEL_DIR = Path("../models")

XGBOOST_RESULTS = MODEL_DIR / "xgb"
MINIROCKET_RESULTS = MODEL_DIR / "minirocket" / "baseline"

METRIC_FILE = "metrics.json"

## Load Model Metrics

In [ ]:

# Load JSONs
with open(XGBOOST_RESULTS / METRIC_FILE, "r") as f:
    xgb_metrics = json.load(f)

with open(MINIROCKET_RESULTS / METRIC_FILE, "r") as f:
    minirocket_metrics = json.load(f)

# Keep them in a dict for easy access later
all_metrics = {
    "xgboost": xgb_metrics,
    "minirocket": minirocket_metrics,
}


## Build Dataframes

In [ ]:
import pandas as pd

# Grab metrics from the nested "details" blocks only, for clean plotting.
# By default we use the "test" split; switch to "val" if you want.

def build_df_overall(all_metrics, split="test"):
    rows = []
    for model, m in all_metrics.items():
        d = m.get(split, {}).get("details", {})
        macro = d.get("macro", {}) or {}
        acc = m.get(split, {}).get("accuracy")
        rows += [
            {"model": model, "metric": "accuracy", "value": acc},
            {"model": model, "metric": "macro_precision", "value": macro.get("precision")},
            {"model": model, "metric": "macro_recall", "value": macro.get("recall")},
            {"model": model, "metric": "macro_f1", "value": macro.get("f1")},
        ]
    df = pd.DataFrame(rows).dropna(subset=["value"])
    order = ["accuracy", "macro_precision", "macro_recall", "macro_f1"]
    df["metric"] = pd.Categorical(df["metric"], categories=order, ordered=True)
    return df.sort_values(["metric", "model"]).reset_index(drop=True)


def build_df_per_class(all_metrics, split="test", classes=None):
    rows = []
    for model, m in all_metrics.items():
        per_cls = m.get(split, {}).get("details", {}).get("per_class", {}) or {}
        keys = classes if classes is not None else list(per_cls.keys())
        for cls in keys:
            blk = per_cls.get(cls, {})
            if not blk:
                continue
            rows += [
                {"model": model, "class": cls, "metric": "precision", "value": blk.get("precision")},
                {"model": model, "class": cls, "metric": "recall",    "value": blk.get("recall")},
                {"model": model, "class": cls, "metric": "f1",        "value": blk.get("f1")},
            ]
    df = pd.DataFrame(rows).dropna(subset=["value"])
    df["metric"] = pd.Categorical(df["metric"], categories=["precision","recall","f1"], ordered=True)
    # Optional: order classes consistently if provided
    if classes is not None:
        df["class"] = pd.Categorical(df["class"], categories=classes, ordered=True)
    return df.sort_values(["class", "metric", "model"]).reset_index(drop=True)



In [ ]:
df_overall_test = build_df_overall(all_metrics, "test")
df_classes_test = build_df_per_class(all_metrics, "test", classes=["ADLs", "Falls", "Near_Falls"])

df_overall_val = build_df_overall(all_metrics, "val")
df_classes_val = build_df_per_class(all_metrics, "val", classes=["ADLs", "Falls", "Near_Falls"])

In [ ]:
df_overall_test

In [ ]:
df_classes_test

In [ ]:
df_overall_val

In [ ]:
df_classes_val

## Plot Graphs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def _annotate_bars(ax):
    import numpy as np
    for p in ax.patches:
        h = p.get_height()
        if np.isfinite(h) and h != 0:
            ax.annotate(f"{h:.3f}",
                        (p.get_x() + p.get_width()/2, h),
                        ha="center", va="bottom", fontsize=9, xytext=(0, 2),
                        textcoords="offset points")

def plot_grouped_bars(df, x_col, group_col, y_col, title, y_max=1.0, x_label_map=None, band_groups=True):
    # Prepare category order & groups
    cats = list(df[x_col].cat.categories if hasattr(df[x_col], "cat") else sorted(df[x_col].unique()))
    groups = sorted(df[group_col].unique())

    x = np.arange(len(cats))
    # Narrower bars
    width = 0.55 / max(len(groups), 1)
    group_span = width * len(groups)

    fig, ax = plt.subplots(figsize=(8, 5))

    # Plot bars per group
    for gi, g in enumerate(groups):
        sub = df[df[group_col] == g]
        y = [sub[sub[x_col] == c][y_col].values[0] if c in set(sub[x_col]) else np.nan for c in cats]
        ax.bar(x + (gi - (len(groups)-1)/2) * width, y, width=width, label=g, zorder=1)

    # Labels & legend
    ax.set_title(title)
    if not band_groups:
        ax.set_ylim(0, y_max if y_max > 0 else 1.0)
    ax.set_ylabel("Score")
    ax.set_xticks(x)
    ax.set_xticklabels([x_label_map.get(c, c) if x_label_map else c for c in cats], rotation=0)
    leg = ax.legend(title="Model", frameon=True, fancybox=True)
    if leg is not None:
        leg.get_frame().set_alpha(0.85)  # subtle background for the legend box

    _annotate_bars(ax)
    ax.grid(axis="y", alpha=0.2)
    plt.tight_layout()
    plt.show()


In [ ]:
overall_label_map = {
    "accuracy": "Accuracy",
    "macro_precision": "Macro Precision",
    "macro_recall": "Macro Recall",
    "macro_f1": "Macro F1",
}

### Plot - Test Data

In [ ]:
plot_grouped_bars(
    df_overall_test,
    x_col="metric",
    group_col="model",
    y_col="value",
    title="Overall Comparison (Test)",
    y_max=1.05,
    x_label_map=overall_label_map,
)

for cls in ["ADLs", "Falls", "Near_Falls"]:
    df_c = df_classes_test[df_classes_test["class"] == cls]
    class_label_map = {"precision": "Precision", "recall": "Recall", "f1": "F1"}
    plot_grouped_bars(
        df_c,
        x_col="metric",
        group_col="model",
        y_col="value",
        title=f"Per-Class Comparison — {cls} (Test)",
        y_max=1.05,
        x_label_map=class_label_map,
    )

### Plot - Validation Data

In [ ]:
plot_grouped_bars(
    df_overall_val,
    x_col="metric",
    group_col="model",
    y_col="value",
    title="Overall Comparison (Validation)",
    y_max=1.05,
    x_label_map=overall_label_map,
)

for cls in ["ADLs", "Falls", "Near_Falls"]:
    df_c = df_classes_val[df_classes_val["class"] == cls]
    class_label_map = {"precision": "Precision", "recall": "Recall", "f1": "F1"}
    plot_grouped_bars(
        df_c,
        x_col="metric",
        group_col="model",
        y_col="value",
        title=f"Per-Class Comparison — {cls} (Validation)",
        y_max=1.05,
        x_label_map=class_label_map,
    )